In this noteboook, we provide an example on how to run the snakemake pipeline on the cluster unix-h7. 

## Steps to get the snakemake pipeline running on h7

1 - Log in to the cluster. To connect via ssh, this is possible with the following command <username>@h7.directory.intra <br>
2 - Clone the git repo: https://github.com/Deltares/interreg-meuse to have the latest code <br>
3 - Create a snakemake environment: `mamba env create -f environment.yml` <br> 
4 - Some part of the bash script (h7_snakefile_cluster.sh file) and snakemake pipeline need to be adapted to your own path <br>


4a - Adapt the snakemake config file. In the file `./config/members_config.yml`, you will need to make sure that the following path are correct:
- if you want to run the unzip rule, make sure you check the name of the main folder. It is assumed that the zipped file is called data.zip
- Make sure the settings for the wflow model are correct 
- `julia_env_fn`: path to your julia environment to run wflow

4b - Adapt some elements of the bash script (h7_snakefile_cluster.sh):
- The bash file has been written so that snakemake first logs in one node from where it will submit all the subsequent runs. Since submitting the job itself does not require a lot of memory nor cores, we select the partition 1vcpu. Each rule however can be run on another partition. This is explained in the next section.To have the script running, you need to adapt the #SBATCH commands to your case: change the --mail-user and --time depending on how long you think your jobs will last. You can have an overview of the partitions at this link: https://publicwiki.deltares.nl/display/Deltareken/Compute+nodes  
- Modify the path to your snakemake environment (line 16). 
- Modify the path to your snakemake script (line 20)

4c - Adapt/Create a config file to define your standard snakemake command line arguments and cluster configurations:
- In your home folder, navigate to ~/.config/snakemake. Here create a folder, named as you wish. In our case, we called it `interreg_daily`. This folder is a specific profile configuration you can call in your bash script. It contains a config.yaml file, a file that is automatically read and recognized by snakemake when we invoke the command `--profile interreg_daily/` in our snakemake command line in the bash script. 
- If you open the ~/.config/snakemake/interreg_daily/config.yaml, you will see that it contains:
    - general snakemake command line arguments such as `restart-times`, `keep-going`, `jobs`, etc. For the meaning of these arguments, you can have a look at the snakemake command line interface: https://snakemake.readthedocs.io/en/stable/executing/cli.html#all-options 
    - general and specific cluster specifications. This includes: 
        - The command submitted every time is defined in the `cluster` section. In this case, that is: `sbatch --qos={resources.qos} --account={resources.account} --time={resources.time} --ntasks={resources.threads} --partition={resources.partition}` The {resources.xxx} are keywords that are defined either from the general `default-resources` section or rule-specific `set-resources` section.
        - defining the general cluster resources in the section `default-resources` that will automatically be called when submitting each snakemake jobs (`--account`, `--time`, etc). 
        - For the rule-specific specification, we use the `set-resources` section. In this part, we first start with the rule it concerns and then add the required argument (`partitiion`, `threads` which can be interpreted as cores, and `time`)

5 - Submit the script. You are now ready to submit your jobs! <br>
However, to make sure you have everything under control, it if advised to first check the following before submitting: <br>
- Make a dry-run of your snakemake pipeline to make sure you get what you expect. Running snakemake as a dry-run will compute the expected number of jobs without submitting them: <br>
    -  `snakemake --unlock -s snakefile --configfile config/members_config.yml` <br>
    -  `snakemake -n -s snakefile --configfile config/members_config.yml --profile interreg_daily/ --wait-for-files --directory $PWD `. You can add the `-q` (it stands for quiet) if you want to only see the table of jobs and not a description of each jobs <br>
    <br>
- Export a figure of your DAG to make sure the rules are correctly ordered: <br>
    -  `snakemake --forceall --dag | dot -Tsvg > dag.svg`   or   `snakemake --rulegraph --dag | dot -Tsvg > dag.svg` <br>

- Submit your job using the command: `sbatch h7_snakefile_cluster.sh`

6 - To check the status of your jobs, you can have a look at the log files. You can also see from the typical slurm commands if your jobs are being properly submitted. An overview of slurm commands can be found here: https://publicwiki.deltares.nl/display/Deltareken/Example%3A+Basic+Slurm+Commands 